In [ ]:
#This is code from the "Build Long-Context AI Apps with Jamba" from DeepLearning.AI that has been modified to read long legistlative documents and answer questions about the bill/act/law from the user
#This code was run inside the imbedded jupyter notebook in the course

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ai21 import AI21Client
from ai21.models.chat import ChatMessage
import uuid
import time

<class 'ModuleNotFoundError'>: No module named 'ai21'

In [ ]:
from utils import get_ai21_api_key
ai21_api_key = get_ai21_api_key()
client = AI21Client(api_key=ai21_api_key)

In [ ]:
#RAG with AI21 Jamba Model in LangChain

In [ ]:
from langchain_ai21 import ChatAI21
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm = ChatAI21(model="jamba-1.5-large",
               max_tokens = 4096,
               temperature = 0.4,
               top_p = 1)

In [ ]:
loader = TextLoader("./inflation_reduction_act_2022.txt")
doc = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400)
documents = text_splitter.split_documents(doc)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = Chroma.from_documents(documents, embedding=embeddings)

In [ ]:
prompt = PromptTemplate.from_template(
    """You are an expert in answering questions based on provided context.
    Answer the question based on the provided context below to the best of your ability.
    The response must be complete, coherent and concise.
    If the answer is not contained in the context, please respond with "answer not in the document"\n
    Here is the context you should use to answer the question: \n
    <context>
    {context}
    </context> \n
    Based on the provided context, answer the following question: {question} \n
    Answer:"""
)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 10})

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
q = "Is this bill hiding anything that could be seen as controversial or favorable to one political party?"

response = rag_chain.invoke(q)
print(f"Answer: {response}")

In [ ]:
docs = retriever.invoke(q)
docs

In [ ]:
questions = ["Does this bill say anything about the prices of drugs and the cost of healthcare?",
             "How will this bill aim to reduce inflation, and how much will it cost?",
             "Is this bill proposing any changes to the federal budget?",
             "Will this affect taxes on the rich, and if so, by how much?
]

for q in questions:
    response = rag_chain.invoke(q)
    print("="*80)
    print(f"Question: {q}")
    print(f"Answer: {response}")

In [3]:
#Here is the output I got when running the program

"""
================================================================================
Question: Does this bill say anything about the prices of drugs and the cost of healthcare?
Answer: Yes, this bill addresses the prices of drugs and the cost of healthcare in the following ways:

- It provides for lower prices for certain high-priced single-source drugs.
- It establishes a maximum fair price for certain drugs, which is subject to an annual percentage increase based on the consumer price index.
- It includes provisions for drug price negotiation program prices to be included in the best price.
- It expands eligibility for low-income subsidies under Medicare Part D.
- It improves access to adult vaccines under Medicaid and CHIP.
- It includes a continued delay of implementation of the prescription drug rebate rule.
================================================================================
Question: How will this bill aim to reduce inflation, and how much will it cost?
Answer: The bill seeks to reduce inflation by increasing direct spending to taxpayers and appropriating significant funds for various initiatives, including clean energy, 
        affordable housing, and conservation efforts. The total appropriations amount to $45,637,400,000, which will remain available until September 30, 2031.
================================================================================
Question: Is this bill proposing any changes to the federal budget?
Answer: Yes, the bill proposes several changes to the federal budget, including appropriations for various government agencies and programs, such as the Internal Revenue Service, 
        the Department of Housing and Urban Development, and the National Park Service. It also includes provisions for the appropriation of funds for specific purposes, such as conservation 
        technical assistance and the improvement of energy or water efficiency in affordable housing.
================================================================================
Question: Will this affect taxes on the rich, and if so, by how much?
Answer: Yes, the amendments will affect taxes on the rich. The increase in research credit against payroll tax for small businesses will not affect the rich. However, 
        the gross-up of direct spending will increase the portion of any payment made to a taxpayer pursuant to an election under section 6417 of the Internal Revenue Code of 1986, 
        or any amount treated as a payment which is made by the taxpayer under subsection (a) of such section, that is direct spending shall be increased by 6.0445 percent. This could affect the rich, 
        as they are more likely to have higher direct spending.
"""

'\n================================================================================\nQuestion: Does this bill say anything about the prices of drugs and the cost of healthcare?\nAnswer: Yes, this bill addresses the prices of drugs and the cost of healthcare in the following ways:\n\n- It provides for lower prices for certain high-priced single-source drugs.\n- It establishes a maximum fair price for certain drugs, which is subject to an annual percentage increase based on the consumer price index.\n- It includes provisions for drug price negotiation program prices to be included in the best price.\n- It expands eligibility for low-income subsidies under Medicare Part D.\n- It improves access to adult vaccines under Medicaid and CHIP.\n- It includes a continued delay of implementation of the prescription drug rebate rule.\n================================================================================\nQuestion: How will this bill aim to reduce inflation, and how much will it cost?\nAn

In [ ]:
vectorstore.delete_collection()